<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - FunkSVD nDCG</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, funksvd
from lenskit.algorithms.funksvd import FunkSVD
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(features,lrate,reg):
    all_recs = []
    test_data = []
    
    features=int(features)
    funksvd = FunkSVD(features=features, iterations=100, lrate=lrate, reg=reg, damping=5, range=None, bias=True) # define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('FunkSVD', funksvd, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
                          
    return results["ndcg"].mean()

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    features = params['features']
    lrate = params['lrate']
    reg = params['reg']
    metric = model_trainer(features,lrate,reg)
    print([features,lrate,reg,metric],',')
    return {'loss': -metric, 'status': STATUS_OK }

space={'features': hp.uniform('features', 1, 100),
       'lrate': hp.uniform('lrate', 0.0005, 0.005),
       'reg': hp.uniform('reg',0.005,0.05)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[62.724909825075464, 0.0038903221640040945, 0.013553716866066857, 0.07120830500967806]                                 
,                                                                                                                      
[93.87549778521898, 0.003048213750287838, 0.02633999621217324, 0.06481498482920438]                                    
,                                                                                                                      
[19.726103734607076, 0.002456337895603651, 0.046694924640317076, 0.08173598362553787]                                  
,                                                                                                                      
[66.0722873737629, 0.0032718753576795206, 0.04065386393155126, 0.06822969427526766]                                    
,                                                                                                                      
[35.50502038093474, 0.002174865452236615

[15.870050985962646, 0.0010835270227804964, 0.01159179683473631, 0.09293631220976573]                                  
,                                                                                                                      
[7.778364904399067, 0.0016559228803461898, 0.008769043528498227, 0.10452786027889742]                                  
,                                                                                                                      
[19.33731377162392, 0.002194951953607767, 0.005260714755475427, 0.08791360945506918]                                   
,                                                                                                                      
[6.4958248128492055, 0.0014369815661296637, 0.006268658702173798, 0.10796560146425285]                                 
,                                                                                                                      
[25.99072369010858, 0.002024421782854833

KeyboardInterrupt: 

In [13]:
array = [[76.2748151055071, 0.0021004798936624723, 0.04006481732721218, 0.05940633636658324]      # 173.05                             
,                                                                                                                      
[18.561440353852284, 0.000634657799987132, 0.026472916031723373, 0.06267570267246766]                                  
,                                                                                                                      
[60.9559193098875, 0.0032737351360976868, 0.018903251389203034, 0.07880284114289621]                                   
,                                                                                                                      
[91.93276619546616, 0.0010895053205585664, 0.030724958631696657, 0.031000237108529265]                                 
,                                                                                                                      
[89.45943252651018, 0.004334375601885162, 0.046318989460861694, 0.058884142045111616]                                  
,                                                                                                                      
[2.5479400657020506, 0.004495640999995894, 0.038523643994574336, 0.09930759553269491]                                  
,                                                                                                                      
[83.25490360571705, 0.0028390411777780527, 0.014478132672591293, 0.07396353944151325]                                  
,                                                                                                                      
[51.6527561461277, 0.00457076490836056, 0.0467887491554247, 0.06985090457376608]                                       
,                                                                                                                      
[23.370471582251664, 0.0010073957897902283, 0.03768455996074605, 0.0683449456774951]                                   
,                                                                                                                      
[18.498412038996804, 0.002491944931826867, 0.01985440763798106, 0.08356981006487746]                                   
,                                                                                                                      
[72.89878615094055, 0.001802892584941934, 0.04766597394406481, 0.051839320657122996]                                   
,                                                                                                                      
[8.846036591335796, 0.0033618583190847366, 0.04272717892704815, 0.09038999359495767]                                   
,                                                                                                                      
[53.370646188341404, 0.0006236999839424912, 0.0392366886735544, 0.03752516270167835]                                   
,                                                                                                                      
[13.185722696684797, 0.0037199675820682528, 0.016582911562732436, 0.0682174078942568],
         [62.724909825075464, 0.0038903221640040945, 0.013553716866066857, 0.07120830500967806]                                 
,                                                                                                                      
[93.87549778521898, 0.003048213750287838, 0.02633999621217324, 0.06481498482920438]                                    
,                                                                                                                      
[19.726103734607076, 0.002456337895603651, 0.046694924640317076, 0.08173598362553787]                                  
,                                                                                                                      
[66.0722873737629, 0.0032718753576795206, 0.04065386393155126, 0.06822969427526766]                                    
,                                                                                                                      
[35.50502038093474, 0.002174865452236615, 0.035614973945023, 0.08688771942814656]                                      
,                                                                                                                      
[18.690615099296316, 0.003386318958682169, 0.015510422943560195, 0.07696086006422286]                                  
,                                                                                                                      
[13.373988087221289, 0.004757693217966355, 0.014990465966154425, 0.06248447217136141]                                  
,                                                                                                                      
[35.51872153665605, 0.002413045197160784, 0.0051658827069823, 0.10040306535417567]                                     
,                                                                                                                      
[32.38075485398548, 0.0025597956487144096, 0.019565009471379496, 0.09356271565326715]                                  
,                                                                                                                      
[27.28764272631521, 0.0026715171809099745, 0.012967478214395003, 0.09737953725929764]                                  
,                                                                                                                      
[36.652824519167694, 0.0013835908121233964, 0.03312967940123915, 0.07876861896385264]                                  
,                                                                                                                      
[59.031180921600104, 0.003253762475204572, 0.016065896438084527, 0.07928114433007705]                                  
,                                                                                                                      
[23.61456607775552, 0.003054938438380013, 0.011775549626006134, 0.09257986615304312]                                   
,                                                                                                                      
[43.54921849106088, 0.0010813812578048037, 0.014152634998222124, 0.08272382175556758]                                  
,                                                                                                                      
[7.041791436679057, 0.0013289791125668663, 0.0050489562600718315, 0.10524544923287511]                                 
,                                                                                                                      
[85.86404911608935, 0.003079590906368224, 0.03271584216171869, 0.06580911379431349]                                    
,                                                                                                                      
[6.800912748670404, 0.0032181449182802427, 0.03157725107384188, 0.09191977848930807]                                   
,                                                                                                                      
[5.28976220001325, 0.0009721536348937866, 0.04210138895164037, 0.08741437030977235]                                    
,                                                                                                                      
[27.317779584352582, 0.0026599761342133376, 0.0052843322177525895, 0.09889638346739713]                                
,                                                                                                                      
[30.775618710116994, 0.0037873010944035387, 0.0439394624806345, 0.08197657954990782]                                   
,                                                                                                                      
[51.17214627723304, 0.0018003354518920623, 0.006397537470035428, 0.10035855076078284]                                  
,                                                                                                                      
[1.2443035723372287, 0.001788359251516856, 0.008416130452815841, 0.10787004034965228]                                  
,                                                                                                                      
[1.4894747490367564, 0.0005181657026522365, 0.022951283734999973, 0.08612787207551954]                                 
,                                                                                                                      
[9.812684544665872, 0.0017351092784145957, 0.008618049880403844, 0.09683371933988642]                                  
,                                                                                                                      
[74.64884314588716, 0.0005422029824245283, 0.01018274693645961, 0.027208281085748905]                                  
,                                                                                                                      
[3.1123288420537136, 0.0019080385701880673, 0.019282446771825078, 0.10120983804133105]                                 
,                                                                                                                      
[13.680806484137582, 0.0013796144630225154, 0.009336773849822746, 0.09608990946554477]                                 
,                                                                                                                      
[42.24884110919313, 0.0009405575056550409, 0.026299192584399285, 0.06342759115474281]                                  
,                                                                                                                      
[16.48691411048565, 0.0014671691358211687, 0.005002082540102469, 0.09785071016984169]                                  
,                                                                                                                      
[2.3129528339793053, 0.0049262532395551555, 0.019621450798211305, 0.0985724078624196]                                  
,                                                                                                                      
[52.31811733875077, 0.000736192838006603, 0.022751038640823867, 0.047566126810812195]                                  
,                                                                                                                      
[98.16796500008833, 0.0022570975816999204, 0.007417752717426273, 0.07661755058981043]                                  
,                                                                                                                      
[73.8824124913664, 0.002072602226412819, 0.04957454430750587, 0.05276685106237504]                                     
,                                                                                                                      
[20.442214438769735, 0.001525333684651294, 0.027587739607118498, 0.08799347826472946]                                  
,                                                                                                                      
[43.754858776811936, 0.004368464609770473, 0.03585861595917058, 0.07371050888959647]                                   
,                                                                                                                      
[88.97566729263335, 0.0036487794424145143, 0.011045495714033548, 0.06766635409839324]                                  
,                                                                                                                      
[9.537666466633706, 0.0011281615782560296, 0.017300213810760648, 0.09589661425225808]                                  
,                                                                                                                      
[59.67808287669614, 0.002842359508750187, 0.025132387544067842, 0.07890001247773748]                                   
,                                                                                                                      
[23.13305536027689, 0.000754979716015727, 0.037827447113027525, 0.05844652219216057]                                   
,                                                                                                                      
[13.425130038360507, 0.0041878730857147235, 0.029419210220134925, 0.07203018243319972]                                 
,                                                                                                                      
[38.54674434192344, 0.0023035353244405888, 0.007434916004137781, 0.10089337775599144]                                  
,                                                                                                                      
[70.02579023782499, 0.0019880782772547876, 0.01811697648235339, 0.0798142458785556]                                    
,                                                                                                                      
[9.061268790642643, 0.0017439128727300495, 0.013402785438058236, 0.09511087979290954]                                  
,                                                                                                                      
[1.1144387387806498, 0.001237953917366133, 0.023883575514258543, 0.10525766294557458]                                  
,                                                                                                                      
[17.894459987816695, 0.001612882376868448, 0.023436730164303442, 0.08713039976253724]                                  
,                                                                                                                      
[82.23929277064744, 0.0028264103401801034, 0.0385721212817702, 0.06230079012813373]                                    
,                                                                                                                      
[47.21104458859031, 0.0011677597500949308, 0.049291132813534436, 0.05446505065231111]                                  
,                                                                                                                      
[30.607495278459083, 0.002483344036805413, 0.02938009466131086, 0.09102933121007316]                                   
,                                                                                                                      
[55.88982565194903, 0.0007169779763758, 0.04667362413661055, 0.03925609365271592]                                      
,                                                                                                                      
[65.26121991174946, 0.0021290443364127883, 0.015834471276188165, 0.08568703659156565]                                  
,                                                                                                                      
[26.610827625473455, 0.0012681949215735946, 0.02139336895006947, 0.09227895247737719]                                  
,                                                                                                                      
[33.37369324635916, 0.0035467938674897045, 0.03154371937465007, 0.08231132106205132]                                   
,                                                                                                                      
[24.555900061466197, 0.000911276361000293, 0.012473998631562667, 0.08522834553852683]                                  
,                                                                                                                      
[1.5565469326784465, 0.0016283821228203071, 0.03591112740904546, 0.10500765446671631]                                  
,                                                                                                                      
[40.11570393822254, 0.0025836698255075185, 0.02090118721728211, 0.08968941539063839]                                   
,                                                                                                                      
[5.449125124528659, 0.002958077034400749, 0.04101378422816161, 0.09432197707048433]                                    
,                                                                                                                      
[21.461503929056754, 0.0005109250949435212, 0.033667857329660836, 0.05482157725661155]                                 
,                                                                                                                      
[15.729833937024413, 0.0018977553237950078, 0.01467579949459181, 0.08322401078634471]                                  
,                                                                                                                      
[11.412649201677723, 0.00235667484881671, 0.024667248773691648, 0.087939577947969]                                     
,                                                                                                                      
[29.70247096327055, 0.0012706287349077033, 0.028382309048002943, 0.08381285148722513]                                  
,                                                                                                                      
[34.57883701503752, 0.0008851411291770697, 0.01723908719123394, 0.07382673992867266]                                   
,                                                                                                                      
[47.584055674969996, 0.003166523706877336, 0.04279008704143831, 0.07840544628441608]                                   
,                                                                                                                      
[81.05621624091472, 0.0033631277975531173, 0.04524897507122033, 0.06245315663938722]                                   
,                                                                                                                      
[3.751963303334799, 0.0018947009458269261, 0.010116565124168156, 0.10409809008917631]                                  
,                                                                                                                      
[1.0869579646982623, 0.0005947188989318597, 0.03419803435658157, 0.08711752170911448]                                  
,                                                                                                                      
[7.513496633169456, 0.0013734219970117873, 0.0060178936262832855, 0.10687116739976067]                                 
,                                                                                                                      
[11.955223297706947, 0.001487702228835057, 0.006987324686953748, 0.09822852366804656]                                  
,                                                                                                                      
[6.87778667270586, 0.0011660122650188824, 0.008519043844220597, 0.1069240061205901]                                    
,                                                                                                                      
[15.870050985962646, 0.0010835270227804964, 0.01159179683473631, 0.09293631220976573]                                  
,                                                                                                                      
[7.778364904399067, 0.0016559228803461898, 0.008769043528498227, 0.10452786027889742]                                  
,                                                                                                                      
[19.33731377162392, 0.002194951953607767, 0.005260714755475427, 0.08791360945506918]                                   
,                                                                                                                      
[6.4958248128492055, 0.0014369815661296637, 0.006268658702173798, 0.10796560146425285]                                 
,                                                                                                                      
[25.99072369010858, 0.002024421782854833, 0.009741155077455512, 0.10411974968784675]                                   
,                                                                                                                      
[4.197869166735606, 0.000705041118816467, 0.013679404261710718, 0.08877134263529601]                                   
,                                                                                                                      
[98.05656800901177, 0.0015267356842901048, 0.007772378397408814, 0.06690613179906202]                                  
,                                                                                                                      
[21.532527536578637, 0.0010535208745557439, 0.011079080901698438, 0.09377208195607153]                                 
,                                                                                                                      
[13.79030939740266, 0.0018131517585699979, 0.012364521809080542, 0.08759011298444756]                                  
,                                                                                                                      
[10.486029727318064, 0.0008312287633605734, 0.016482094095291118, 0.08536205737469245]                                 
,                                                                                                                      
[37.631533698378384, 0.0009914781248418015, 0.01880444005477934, 0.07710487437812252]                                  
,                                                                                                                      
[18.23775603957124, 0.0024725490112849323, 0.008197539981318119, 0.08176206234286819]                                  
,                                                                                                                      
[28.425137920815494, 0.002736110888700695, 0.005821150054397378, 0.0969880422584462]                                   
,                                                                                                                      
[6.077403732079077, 0.0006025962648560283, 0.02053899122810702, 0.0801191781767002]                                    
,                                                                                                                      
[24.047089119720752, 0.0014019729381804463, 0.014628279892141603, 0.10224068203878917]                                 
,                                                                                                                      
[31.574349812508792, 0.002148737523088361, 0.006729864755135852, 0.10511225906261217]                                  
,                                                                                                                      
[14.764455861591912, 0.001173107671026565, 0.02232802372062334, 0.08522536380348276]                                   
,                                                                                                                      
[41.46966923493241, 0.003999723624469862, 0.008941166147976552, 0.07598605187829621]]

In [14]:
metric = pd.DataFrame(array)
metric.columns = ['features','learning rate','reg','nDCG']
metric.head(5)

,features,learning rate,reg,nDCG
0,76.274815,0.002100,0.040065,0.059406
1,18.561440,0.000635,0.026473,0.062676
2,60.955919,0.003274,0.018903,0.078803
3,91.932766,0.001090,0.030725,0.031000
4,89.459433,0.004334,0.046319,0.058884


In [7]:
metric.sort_values(by=['nDCG'],ascending=False).head(10)

,features,learning rate,reg,nDCG
85,6.495825,0.001437,0.006269,0.107966
35,1.244304,0.001788,0.008416,0.107870
81,6.877787,0.001166,0.008519,0.106924
79,7.513497,0.001373,0.006018,0.106871
57,1.114439,0.001238,0.023884,0.105258
28,7.041791,0.001329,0.005049,0.105245
97,31.574350,0.002149,0.006730,0.105112
67,1.556547,0.001628,0.035911,0.105008
83,7.778365,0.001656,0.008769,0.104528
86,25.990724,0.002024,0.009741,0.104120


In [5]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\1M\\FunkSVD\\results\\metric_nDCG.csv')
metric

,features,learning rate,reg,nDCG
0,76.274815,0.002100,0.040065,0.059406
1,18.561440,0.000635,0.026473,0.062676
2,60.955919,0.003274,0.018903,0.078803
3,91.932766,0.001090,0.030725,0.031000
4,89.459433,0.004334,0.046319,0.058884
...,...,...,...,...
95,6.077404,0.000603,0.020539,0.080119
96,24.047089,0.001402,0.014628,0.102241
97,31.574350,0.002149,0.006730,0.105112
98,14.764456,0.001173,0.022328,0.085225


In [6]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
y = metric['learning rate']
z = metric['features'] 
c = metric['nDCG']


fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(x, y, z, c=c, cmap=plt.hsv())
ax.set_xlabel('reg')
ax.set_ylabel('learning rate')
ax.set_zlabel('features')
fig.colorbar(img)
plt.show()